In [1]:
import os
import random
import shutil
import json
import random

In [28]:
sp = 'validation'
path = 'e:/p2_data/120personal_veh/01data/2Validation/'+sp+'_json/'+sp+'2_ann.json'

with open(path, 'r') as file:
    data = json.load(file)

classes = ['Motorcycle_Pedestrian Road Violation', 'Motorcycle_No Helmet', 'Motorcycle_Jaywalking', 
'Motorcycle_Signal Violation', 'Motorcycle_Stop Line Violation', 'Motorcycle_Crosswalk Violation', 
'Bicycle_Pedestrian Road Violation', 'Bicycle_No Helmet', 'Bicycle_Jaywalking', 
'Bicycle_Signal Violation', 'Bicycle_Stop Line Violation', 'Bicycle_Crosswalk Violation', 
'Kickboard_Pedestrian Road Violation', 'Kickboard_No Helmet', 'Kickboard_Jaywalking', 
'Kickboard_Signal Violation', 'Kickboard_Stop Line Violation', 'Kickboard_Crosswalk Violation', 'Kickboard_Passenger Violation']


annotations = data['annotations']
maxlen = len(annotations)
searched_list=[]
ban_list = ['T008037_072_0012_C_D_F_0.jpg',
'T008037_067_0096_C_D_F_0.jpg',
'T008037_072_0156_C_D_F_0.jpg',
'T008037_009_0020_C_D_F_0.jpg',
'T008037_068_0040_C_D_F_0.jpg', 
'T008037_068_0040_C_D_F_0.jpg',
'T008037_068_0040_C_D_F_0.jpg',
'T015350_007_0378_C_D_F_1.jpg',
'T008037_100_0060_C_D_F_0.jpg',
'T008037_066_0126_C_D_F_0.jpg',
'T008037_066_0126_C_D_F_0.jpg',
'T008037_067_0160_C_D_F_0.jpg',
'T008037_066_0144_C_D_F_0.jpg',
'T008037_066_0144_C_D_F_0.jpg',
'T008034_001_0144_B_D_F_0.jpg', 
'T008037_068_0100_C_D_F_0.jpg', 
'T008037_068_0100_C_D_F_0.jpg',
'T007931_000_0070_B_D_F_0.jpg']

to_save_dict = {i:[] for i in range(len(classes))}
tot_dict = {i:[] for i in range(len(classes))}

for i in range(maxlen):
    idx = random.randint(0,maxlen-1)
    label = annotations[idx]['category_id']
    img_id = annotations[idx]['image_id']
    if (img_id in searched_list):
        continue
    else:
        filename = data['images'][img_id-1]['file_name']
        if filename in ban_list:
            continue
        else:
            tot_dict[label].append([data['images'][img_id-1],annotations[idx]])
            searched_list.append(img_id)

for i in range(16):
    print(i,len(tot_dict[i]))
    to_save_dict[i] = random.sample(tot_dict[i],10)
for i in range(17,19):
    print(i,len(tot_dict[i]))
    to_save_dict[i] = random.sample(tot_dict[i],10)


with open('sample_data.json', 'w') as file:
    json.dump(to_save_dict,file)


17 137
18 103


In [30]:
sp = 'train'
path = 'e:/p2_data/120personal_veh/01data/2Validation/'+sp+'_json/'+sp+'2_ann.json'

with open(path, 'r') as file:
    data = json.load(file)

classes = ['Motorcycle_Pedestrian Road Violation', 'Motorcycle_No Helmet', 'Motorcycle_Jaywalking', 
'Motorcycle_Signal Violation', 'Motorcycle_Stop Line Violation', 'Motorcycle_Crosswalk Violation', 
'Bicycle_Pedestrian Road Violation', 'Bicycle_No Helmet', 'Bicycle_Jaywalking', 
'Bicycle_Signal Violation', 'Bicycle_Stop Line Violation', 'Bicycle_Crosswalk Violation', 
'Kickboard_Pedestrian Road Violation', 'Kickboard_No Helmet', 'Kickboard_Jaywalking', 
'Kickboard_Signal Violation', 'Kickboard_Stop Line Violation', 'Kickboard_Crosswalk Violation', 'Kickboard_Passenger Violation']


annotations = data['annotations']
maxlen = len(annotations)
searched_list=[]
ban_list = ['T008037_072_0012_C_D_F_0.jpg',
'T008037_067_0096_C_D_F_0.jpg',
'T008037_072_0156_C_D_F_0.jpg',
'T008037_009_0020_C_D_F_0.jpg',
'T008037_068_0040_C_D_F_0.jpg', 
'T008037_068_0040_C_D_F_0.jpg',
'T008037_068_0040_C_D_F_0.jpg',
'T015350_007_0378_C_D_F_1.jpg',
'T008037_100_0060_C_D_F_0.jpg',
'T008037_066_0126_C_D_F_0.jpg',
'T008037_066_0126_C_D_F_0.jpg',
'T008037_067_0160_C_D_F_0.jpg',
'T008037_066_0144_C_D_F_0.jpg',
'T008037_066_0144_C_D_F_0.jpg',
'T008034_001_0144_B_D_F_0.jpg', 
'T008037_068_0100_C_D_F_0.jpg', 
'T008037_068_0100_C_D_F_0.jpg',
'T007931_000_0070_B_D_F_0.jpg']

to_save_dict = {i:[] for i in range(len(classes))}
tot_dict = {i:[] for i in range(len(classes))}

for i in range(maxlen):
    idx = random.randint(0,maxlen-1)
    label = annotations[idx]['category_id']
    img_id = annotations[idx]['image_id']
    filename = data['images'][img_id-1]['file_name']
    if filename in ban_list:
        continue
    else:
        tot_dict[label].append([data['images'][img_id-1],annotations[idx]])
        searched_list.append(img_id)
print(len(tot_dict[16]))

0
